In [54]:
import pandas as pd

## GBIF Dataset
We clean the dataset by removing unnecessary columns and removing all NA values as well as duplicate rows. We also only keep mushrooms for the genus we have pictures of. Since Kingdom, Phylum, and Class all have the same value at the end of the clean, we remove those columns as well since they only enlarge the data unnecessarily.

In [55]:
gbif: pd.DataFrame = pd.read_csv('./data/gbif.csv')

/var/folders/12/w6sy18kn02z3rqybsvv_x84r0000gn/T/ipykernel_68656/2382710439.py:1: DtypeWarning: Columns (45) have mixed types. Specify dtype option on import or set low_memory=False.
  gbif: pd.DataFrame = pd.read_csv('./data/gbif.csv')


In [56]:
drop: list = ["gbifID", "datasetKey", "occurrenceID", "verbatimScientificNameAuthorship", "stateProvince", 
"occurrenceStatus", "individualCount", "publishingOrgKey", "coordinatePrecision", "elevation", "elevationAccuracy", 
"depth", "depthAccuracy", "basisOfRecord", "institutionCode", "collectionCode", "catalogNumber", "recordNumber", 
"identifiedBy", "dateIdentified", "license", "rightsHolder", "recordedBy", "typeStatus", "establishmentMeans", 
"lastInterpreted", "mediaType", "issue", "coordinateUncertaintyInMeters", "eventDate", "day", "month", "year", "speciesKey", "taxonKey"]
gbif.drop(drop, axis = 1, inplace = True)

In [57]:
gbif = gbif.dropna()
gbif = gbif.drop_duplicates()

In [58]:
keep: list = ['Amanita', 'Cortinarius', 'Russula', 'Entoloma', 'Hygrocybe', 'Lactarius', 'Suillus']
gbif = gbif[gbif.genus.isin(keep)]
gbif = gbif.reset_index(drop = True)
gbif['uniqueID'] = gbif.index

In [59]:
gbif.head()

,kingdom,phylum,class,order,family,genus,species,infraspecificEpithet,taxonRank,scientificName,verbatimScientificName,countryCode,locality,decimalLatitude,decimalLongitude,uniqueID
0,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Amanitaceae,Amanita,Amanita rubescens,rubescens,VARIETY,Amanita rubescens var. rubescens,Amanita rubescens var. rubescens (Pers.) Gray,DK,Lundsgård Storskov,55.43561,10.68910,0
1,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Amanitaceae,Amanita,Amanita rubescens,rubescens,VARIETY,Amanita rubescens var. rubescens,Amanita rubescens var. rubescens (Pers.) Gray,DK,Grevsmose,57.27064,10.19545,1
2,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Amanitaceae,Amanita,Amanita rubescens,rubescens,VARIETY,Amanita rubescens var. rubescens,Amanita rubescens var. rubescens (Pers.) Gray,DK,Torstedlund Skov,56.79856,9.75412,2
3,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Amanitaceae,Amanita,Amanita rubescens,rubescens,VARIETY,Amanita rubescens var. rubescens,Amanita rubescens var. rubescens (Pers.) Gray,DK,Grønholt Hegn,55.93501,12.38527,3
4,Fungi,Basidiomycota,Agaricomycetes,Agaricales,Amanitaceae,Amanita,Amanita vaginata,vaginata,VARIETY,Amanita vaginata var. vaginata,Amanita vaginata var. vaginata (Bull.) Fr.,DK,Store Hareskov,55.77313,12.41414,4


In [60]:
gbif['kingdom'].value_counts()

Fungi    12188
Name: kingdom, dtype: int64

In [61]:
gbif['phylum'].value_counts()

Basidiomycota    12188
Name: phylum, dtype: int64

In [62]:
gbif['class'].value_counts()

Agaricomycetes    12188
Name: class, dtype: int64

In [63]:
gbif['taxonRank'].value_counts()

VARIETY    12067
FORM         121
Name: taxonRank, dtype: int64

In [64]:
locations: pd.DataFrame = gbif[['uniqueID', 'decimalLatitude', 'decimalLongitude']]
locations.to_csv('./data/cleaned/locations.csv')

# The entire database is
# Kingdom: Fungi
# Phylum: Basidiomycota
# Class: Agaricomycetes
drop = ['kingdom', 'phylum', 'class', 'decimalLatitude', 'decimalLongitude', 'taxonRank']
gbif.drop(drop, axis = 1, inplace = True)
gbif.to_csv('./data/cleaned/gbif.csv')

In [16]:
import sqlite3
import os

In [8]:
connection = sqlite3.connect("./databases/images.db")
cursor = connection.cursor()
cursor.execute("CREATE TABLE images(genus string, img blob)")
connection.commit()

In [11]:
path = "./data/Mushrooms/"
for directory in os.listdir(path):
    if directory not in keep:
        continue
    
    for filename in os.listdir(f"{path}/{directory}/"):
        image = open(f"{path}/{directory}/{filename}", 'rb').read()
        connection.execute("INSERT INTO images VALUES(?,?)", (directory, sqlite3.Binary(image)))
        connection.commit()

connection.close()